In [2]:
import time
import random
import pandas as pd
from content_cleaning import *

In [5]:
# 初始化处理器
processor = XHSMixedLanguageProcessor(cache_size=2000, max_workers=12)
# Load the cooked contents data
cont = pd.read_json('..\..\Data\processed\contents_cooked.json')

In [ ]:

# # 示例笔记
# sample_notes = [
#     "今天去了Dallas的鲜芋仙 #美食打卡# 这家MeetFresh真的超级好吃！The taro balls were amazing! 强烈推荐大家尝试～",
#     "新开的台湾甜品店@小红书美食博主 服务态度nice，芋圆Q弹，仙草冻很香 #DFW美食# [笑哭R] 2001 Coit Rd真的很方便",
#     "约上闺蜜一起去吃甜品，牛奶冰+芋圆组合👍 The dessert was incredibly refreshing on such a hot day! https://xiaohongshu.com/...",
#     "今日份打卡：鲜芋仙 Park Pavillion Center，人均$15左右，店内环境整洁，服务态度很好，definitely worth the price!",
# ]

# processed_text = processor.process_text(sample_notes[0], enable_translation=True)
# print("单条处理结果:", processed_text)
# processed_texts = processor.batch_process(sample_notes)
# print("\n批量处理结果:")
# for i, text in enumerate(processed_texts):
#     print(f"{i+1}. {text}")

# combine the title and note_body into a single string
def process_text(note):
    return note['title'] + ' ' + note['note_body']

# Apply the function to the DF
cont['text'] = cont.apply(process_text, axis=1).astype(str)

# Apply the batch processing function to the DF.text column
start_time = time.time()
processed_texts = processor.batch_process(cont['text'].tolist(), enable_translation=True)
end_time = time.time()

print(f"批量处理耗时: {end_time - start_time:.2f}秒")

# save the processed texts to a new column in the DF
cont['semantic_proc_text'] = processed_texts
# remove the original text column
cont.drop(columns=['text'], inplace=True)
# save the processed DF to a new JSON file
cont.to_json('..\..\Data\processed\contents_cooked_semantic.json', orient='records', lines=True, force_ascii=False)


In [3]:
cont.columns

Index(['note_id', 'user_id', 'title', 'note_body', 'tag_list', 'image_count',
       'content_type_video', 'hot_note', 'post_time', 'last_update_time',
       'scraped_time', 'elapsed_time', 'liked_count', 'collected_count',
       'comment_count', 'share_count', 'interaction_count',
       'semantic_proc_text'],
      dtype='object')

In [4]:
processor.get_stats()

{'cache_size': 1965,
 'cache_capacity': 2000,
 'cache_hits': 176,
 'translation_requests': 16883,
 'cache_hit_rate': 0.01042468755552923}

In [17]:
# 读取处理后的数据
cont = pd.read_json('..\..\Data\processed\contents_cooked_semantic.json', lines=True)

# 随机挑3个笔记输出标题+正文, 并和处理后的文本对比
for i in range(3):
    idx = random.randint(0, len(cont) - 1)
    print(f"原始文本: {cont.iloc[idx]['title']} {cont.iloc[idx]['note_body']}")
    print(f"处理后文本: {cont.iloc[idx]['semantic_proc_text']}")
    print("-" * 80)

原始文本: 攻略|小众唯美的拍摄地｜融创海洋世界 趁暑期特惠好便宜的票，和小姐妹去了#融创海世界[话题]# 海洋隧道的天花板，超美！不仅适合带儿童的亲子出游，女生们来拍照都特别浪漫唯美哦[黄金薯R]
	
这次我去的是无锡，南昌也有一家。
暑假两家都有优惠，大家可以网上看看对比下票价。参考票价 成人夜场79元左右，白天170（暑假还是夜场合算 白天的话，非暑假会便宜点）
	
里面不仅有海洋生物，还有各种萌宠和热带爬行动物，游乐项目大部分是免费的，没带儿童的大人都可以坐！门口有给你免费合影（送一张小张，如果洗大张要额外收费哦）
	
暑假夜场票是很值得 比普通票价便宜很多，晚上时间完全够玩。
#海洋馆[话题]##南昌融创海世界[话题]#
	
#无锡融创海世界[话题]# #海洋馆拍照[话题]#
处理后文本: 攻略 小众唯美的拍摄地 融创海洋世界 趁暑期特惠好便宜的票,和小姐妹去了融创海世界 海洋隧道的天花板,超美!不仅适合带儿童的亲子出游,女生们来拍照都特别浪漫唯美哦 这次我去的是无锡,南昌也有一家 暑假两家都有优惠,大家可以网上看看对比下票价 参考票价 成人夜场79元左右,白天170(暑假还是夜场合算 白天的话,非暑假会便宜点) 里面不仅有海洋生物,还有各种萌宠和热带爬行动物,游乐项目大部分是免费的,没带儿童的大人都可以坐!门口有给你免费合影(送一张小张,如果洗大张要额外收费哦) 暑假夜场票是很值得 比普通票价便宜很多,晚上时间完全够玩 海洋馆南昌融创海世界 无锡融创海世界 海洋馆拍照
--------------------------------------------------------------------------------
原始文本: 一生必去西班牙托莱多🇪🇸世界文化遗产 欧洲历史名城，梦幻中的哥特式古城，托莱多，浪漫而又神秘
【景点攻略】
亮点特色:
托莱多(Toledo)是西班牙的古城。它始于罗马时期，在腓力二世前为卡斯蒂利亚王国(后西班牙王国)首都，为世界文化遗产之一。
详细地址:
托莱多市位于马德里以南70公里处，塔霍河环抱着半个城市，以艺术品著名。这里的雕塑品和金属饰品堪称精美。市内建筑及布局具有典型西班牙风格，被定为国家文物保护单位。托莱多为欧洲历史名城，公元前192年被罗马人占领，公元527年西哥特人统治西班牙
依山而建，所以建议各位小